Libraries

In [ ]:
# Libraries
import numpy as np
import sys
import os
import json
from pathlib import Path
import re
import litellm 
from pydantic import BaseModel
import pandas as pd

# add path to the dataset entities
sys.path.append(os.path.abspath("../0. Helpers"))
sys.path.append(os.path.abspath("../2. Data Processing/_dataset_entities"))

from datasetProcessing import Entity, recursive_fix
from reflection_helpers import get_token_context_include, get_token_context_exclude, get_entity_context

Settings

In [ ]:
# potential_tokens_folder = "paper"
# prob_threshold = 0.95

potential_tokens_folder = "adapted"

Topic

In [ ]:
all_configs = {
    "ai": 10,
    "literature": 10,
    "music": 10,
    "politics": 20,
    "science": 20,
    "multinerd_en": 20,
    "multinerd_pt": 20,
    "ener": 20,
    "lener": 20,
    "neuralshift": 20
}

Process functions

In [ ]:
def process_token(topic, token_with_context):

    # Search for examples where token is inside
    classification_path = Path(f"classification/{topic}/train/data")

    positive_examples = []

    # classification files
    for file in classification_path.glob("*.json"):

        with open(file, mode='r', encoding="utf-8") as f:
            content = f.read()
            
        content = re.sub(r',\s*$', '', content)
        data = json.loads(content)

        true_entities = data["true_entities"]
        entity_tokens = data["classification"]["entity"]
        context_tokens = data["classification"]["context"]
        
        # check for context
        for token in token_with_context["context_high_prob"]:

            # candidate if in entity or context tokens
            if token in entity_tokens or token in context_tokens:

                for entity in true_entities:
                    if token in entity["span"]:
                        context = get_entity_context(data["sentence"], entity["span"], context_length = 4)
                        
                        if context:
                            demonstration = {
                                "sentence": f"...{' '.join(context)}...",
                                "entity": entity["span"]
                            }
                        else:
                            print(topic)
                            print(file.name)
                            print(token)
                            raise ValueError("No context found")

                        positive_examples.append(demonstration)

    token_json = {
        "token": token,
        "positive_examples": positive_examples,
    }
    
    return token_json

In [ ]:
def identify_unseen(tokens, llm_entities, probs_dict):

    unseen_tokens = []
    keys_lower = {key.lower() for key in probs_dict.keys()}

    # check for unseen tokens
    for idx, token in enumerate(tokens):
        if token.lower() not in keys_lower:

            # check if token is not inside a predicted entity
            if all(token.lower() not in e["span"].lower() for e in llm_entities) and token not in unseen_tokens:

                context = get_token_context_exclude(tokens, idx)
            
                # check if tokens on context have high probability of being entity/context
                context_high_prob = []
                context_not_inside_entity = []

                for t in context:

                    # paper vs adapted!
                    if potential_tokens_folder == "adapted":
                        high_prob = t in probs_dict and (probs_dict[t]["prob_e"] == 1 or probs_dict[t]["prob_c"] == 1)
                    elif potential_tokens_folder == "paper" and prob_threshold:
                        high_prob = t in probs_dict and (probs_dict[t]["prob_e"] >= prob_threshold or probs_dict[t]["prob_c"] >= prob_threshold)
                    else:
                        raise ValueError("Invalid potential_tokens_folder")
                    
                    not_inside_entity = all(t.lower() not in e["span"].lower() for e in llm_entities)
    
                    context_high_prob.append(high_prob)
                    context_not_inside_entity.append(not_inside_entity)

                # if ANY token in context has high prob and ALL are not inside entity
                if any(context_high_prob) and all(context_not_inside_entity):

                    unseen_tokens.append({
                        "token": token,
                        "context": context,
                        "context_high_prob": [s for s, f in zip(context, context_high_prob) if f]
                    })

    return unseen_tokens

In [ ]:
def process_instance(topic, file_path, probs_dict):
    
    with open(file_path, mode='r', encoding="utf-8") as f:
        content = f.read()

    if not content.strip():
        print(f"🗑️ Empty file detected: {file_path}")
        return None

    # Fix JSON extra comma
    content = re.sub(r',\s*$', '', content)
    data = json.loads(content)

    # Apply encoding fix
    data = recursive_fix(data)  

    # extract entities
    tokens = data["tokens"]
    # true_entities = data.get("true_entities", [])
    llm_entities = data["entities"]
    sentence = data["sentence"]

    unseen_tokens = identify_unseen(tokens, llm_entities, probs_dict)
    instance_tokens_info = []

    if unseen_tokens:
        
        print(f"Found {len(unseen_tokens)} unseen tokens in {file_path}")
        print("Sentence: ", sentence)
        print(unseen_tokens)
        print("\n---\n")

        for token_with_context in unseen_tokens:
            token_with_examples = process_token(topic, token_with_context)

            # only keep tokens with more than 1 positive example
            if len(token_with_examples["positive_examples"]) > 1:
                instance_tokens_info.append(token_with_examples)

        if instance_tokens_info:
            instance_output_json = {
                "sentence": sentence,
                "unseen_tokens": instance_tokens_info
            }

            # save token to a file
            output_path = f"error_reflection/unseen/{topic}/{potential_tokens_folder}/{Path(file_path).stem}.json"
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(json.dumps(instance_output_json, ensure_ascii=False, indent=4))

    return instance_tokens_info

Run for all configs

In [ ]:
unseen_summary = {}

for topic, n in all_configs.items():

    print(f"Processing topic: {topic} with top {n} demos")

    best_results_folder = f"results/demo_type/{topic}/in_context_top{n}"

    # ensure folder exists
    os.makedirs(f"error_reflection/unseen/{topic}/{potential_tokens_folder}", exist_ok=True)

    # load probs dict
    probs_path = f"classification/{topic}/train/_probs.json"

    # read file
    with open(probs_path, "r", encoding="utf-8") as f:
        probs_dict = json.load(f)

    # Process all instances in the folder
    instances = 0
    instances_with_unseen = 0

    total_unseen_tokens = 0

    for file_path in Path(best_results_folder).glob("*.json"):
        instances += 1

        unseen_tokens = process_instance(topic, file_path, probs_dict)
        
        if unseen_tokens:
            instances_with_unseen += 1
            total_unseen_tokens += len(unseen_tokens)

    unseen_summary[topic] = {
        "instances": instances,
        "instances_with_unseen": instances_with_unseen,
        "instances_with_unseen_percent": instances_with_unseen / instances * 100,
        "total_unseen_tokens": total_unseen_tokens,
    }   

In [ ]:
for topic, summary in unseen_summary.items():
    print(f"Topic: {topic}")
    print(f"Instances with unseen: {summary['instances_with_unseen']} ({summary['instances_with_unseen_percent']:.1f}%)")
    print(f"Total unseen tokens: {summary['total_unseen_tokens']}")

    print("\n---")